##### Ollama Models
gemma2:2b                                                        
llama3.1

In [40]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

In [22]:
from langchain_community.llms import Ollama
llm = Ollama(model='gemma2:2b')

Invoking directly with LLM

In [23]:
llm.invoke('What is your name')

'My name is Gemma. 😊  How can I help you today? \n'

Invoking with messages

In [43]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]
llm.invoke(messages)

'French: Bonjour, comment allez-vous ? \n'

Invoking LLM with Prompt and Chaining

In [24]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ('system','You are an expert AI Engineer. Provide me answers based on the questions'),
        ('user','{input}')
    ]
)

chain = prompt|llm
response = chain.invoke({'input':'Tell me about Langsimth?'})
response

'Okay, let\'s dive into the world of LangChain! \n\n**What is LangChain?**\n\nLangchain is a powerful framework designed to simplify and streamline the development of applications powered by large language models (LLMs). Think of it as the Swiss Army knife for building LLM-based AI solutions.  Here\'s why it\'s so popular:\n\n* **Abstraction:** LangChain handles complex tasks like managing prompts, retrieving data, and interacting with LLMs. This lets you focus on the core logic of your applications without getting bogged down in low-level details.\n* **Modular Design:** You can assemble various components like chains (sequences of operations), memory (to store and recall information), and agents (for autonomous decision making). \n* **Extensible:** LangChain encourages flexibility, letting you easily integrate with different LLMs and data sources (think databases, APIs, etc.).\n\n**Why is it so exciting?**\n\nLangChain empowers developers to build a wide range of innovative applicatio

Chaining with output parser

In [25]:
from langchain_core.output_parsers import StrOutputParser
output_parser=StrOutputParser()
chain=prompt|llm|output_parser
response= chain.invoke('What are Gans?')
response

'**Gans (Generative Adversarial Networks)** are a type of deep learning model that combines two neural networks, a **generator** and a **discriminator**, to create realistic synthetic data. \n\nHere\'s how they work:\n\n* **Generator:** This network takes random noise as input and attempts to create an output resembling real-world data like images, videos, or text.\n* **Discriminator:** This network receives both real and generated data (from the generator) and tries to differentiate between them. The discriminator assigns a score based on how well the generated data looks similar to real data. \n\nThe training process:\n\n1. **Training the Discriminator:**  The discriminator is trained to label each input as either real or fake. Through backpropagation, it learns to identify subtle differences between genuine and synthetic data.\n2. **Training the Generator:** The generator tries to create data that closely mimics what the discriminator labels as "real". It receives feedback from the 

Using Streamlit

In [ ]:
# import os
# from dotenv import load_dotenv

# from langchain_community.llms import Ollama
# import streamlit as st
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.output_parsers import StrOutputParser

# load_dotenv()

# ## Langsmith Tracking
# os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
# os.environ["LANGCHAIN_TRACING_V2"]="true"
# os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

# ## Prompt Template
# prompt=ChatPromptTemplate.from_messages(
#     [
#         ("system","You are a helpful assistant. Please respond to the question asked"),
#         ("user","Question:{question}")
#     ]
# )

# ## streamlit framework
# st.title("Langchain Demo With Gemma Model")
# input_text=st.text_input("What question you have in mind?")


# ## Ollama Llama2 model
# llm=Ollama(model="gemma:2b")
# output_parser=StrOutputParser()
# chain=prompt|llm|output_parser

# if input_text:
#     st.write(chain.invoke({"question":input_text}))

Chains and Invoking by giving our custom content

In [26]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [27]:
web_data = WebBaseLoader('https://docs.smith.langchain.com/tutorials/Administrators/manage_spend').load()
split = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
split_web_data = split.split_documents(web_data)

In [28]:
embeddings = OllamaEmbeddings(model='gemma2:2b')
vect_embed_data = FAISS.from_documents(split_web_data,embeddings)

Retrival Chain, Document Chain

In [29]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    
    """
)

document_chain = create_stuff_documents_chain(llm,prompt)
# document_chain

In [30]:
from langchain_core.documents import Document
document_chain.invoke({
    'input':"What are LangSmith's two usage limits?",
    'context':[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

"What are LangSmith's two usage limits?  \n"

In [31]:
from langchain.chains import create_retrieval_chain
retriever = vect_embed_data.as_retriever()
ret_chain = create_retrieval_chain(retriever,document_chain)
response=ret_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})

In [32]:
response

{'input': 'LangSmith has two usage limits: total traces and extended',
 'context': [Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='meaning we log around 100,000-130,000 traces per dayExpected Growth in Load: We expect to double in size in the near future.From these assumptions, we can do a quick back-of-the-envelope calculation to get a good limit of:limit = current_load_per_day * expected_growth * days/month      = 130,000 * 2 * 30      = 7,800,000 traces / monthWe click on the edit icon on the right side of the table for our Prod row, and can enter this limit as follows:noteWhen set without the extended data retention traces limit, the maximum cost estimator assumes that all traces are using extended data retention.Cutting maximum spend wit

In [33]:
response['answer']

"The provided context outlines a workflow for managing data retention and usage limits in the context of an AI application, specifically LangSmith.  Here's a breakdown:\n\n**Understanding Costs & Limits:**\n\n* **High Usage Costs:** The initial cost estimate for extended data retention is $40,000 per month, which suggests that data retention features can be expensive. \n* **Data Retention Strategy:**  They utilize an 'extended data retention' strategy where they keep around 10% of traces older than 14 days for analysis and historical use. This limits costs significantly, as the full dataset is not retained.\n* **Usage Limits:** They implemented usage limits to prevent unexpected spending. A limit of ~7.5k per month was achievable by keeping only ~10% of traces (representing a lower volume than the potential cost) for extended retention.\n\n**How to Effectively Set Usage Limits:**\n\n* **Assumptions:** It's crucial to clearly define assumptions about usage patterns, like their AI applic